# Predictive Maintenance

In this example, we are trying to estimate the remaining lifetime of mechanical parts.

**Loading csv files**

In [1]:
MNT <- read.csv("data/DATA_4.03_MNT.csv")
head(MNT)

,lifetime,broken,pressureInd,moistureInd,temperatureInd,team,provider
,<int>,<int>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
1,56,0,92.17885,104.23020,96.51716,TeamA,Provider4
2,81,1,72.07594,103.06570,87.27106,TeamC,Provider4
3,60,0,96.27225,77.80138,112.19617,TeamA,Provider1
4,86,1,94.40646,108.49361,72.02537,TeamC,Provider2
5,34,0,97.75290,99.41349,103.75627,TeamB,Provider1
6,30,0,87.67880,115.71226,89.79210,TeamA,Provider1


**Exploring the "MNT" Data**

In [2]:
str(MNT)

'data.frame':	1000 obs. of  7 variables:
 $ lifetime      : int  56 81 60 86 34 30 68 65 23 81 ...
 $ broken        : int  0 1 0 1 0 0 0 1 0 1 ...
 $ pressureInd   : num  92.2 72.1 96.3 94.4 97.8 ...
 $ moistureInd   : num  104.2 103.1 77.8 108.5 99.4 ...
 $ temperatureInd: num  96.5 87.3 112.2 72 103.8 ...
 $ team          : Factor w/ 3 levels "TeamA","TeamB",..: 1 3 1 3 2 1 2 2 2 3 ...
 $ provider      : Factor w/ 4 levels "Provider1","Provider2",..: 4 4 1 2 1 1 2 3 2 4 ...


In [3]:
summary(MNT)

    lifetime        broken       pressureInd      moistureInd    
 Min.   : 1.0   Min.   :0.000   Min.   : 33.48   Min.   : 58.55  
 1st Qu.:34.0   1st Qu.:0.000   1st Qu.: 85.56   1st Qu.: 92.77  
 Median :60.0   Median :0.000   Median : 97.22   Median : 99.43  
 Mean   :55.2   Mean   :0.397   Mean   : 98.60   Mean   : 99.38  
 3rd Qu.:80.0   3rd Qu.:1.000   3rd Qu.:112.25   3rd Qu.:106.12  
 Max.   :93.0   Max.   :1.000   Max.   :173.28   Max.   :128.60  
 temperatureInd      team          provider  
 Min.   : 42.28   TeamA:336   Provider1:254  
 1st Qu.: 87.68   TeamB:356   Provider2:266  
 Median :100.59   TeamC:308   Provider3:242  
 Mean   :100.63               Provider4:238  
 3rd Qu.:113.66                              
 Max.   :172.54                              

In this dataset, about 40% of the pieces are broken. For example, about 336 pieces have been allocated to teamA, and 254 pieces has been supplied by Provider1.

### Linear Regression Model

Let's build a linear regression model, in order to estimate the lifetime of a pieces ('lifetime') based on the other variables of the dataset (minus the 'broken' variable):

In [4]:
# Build a Linear Regression Model
linregmodel <- lm(lifetime~.-broken,data=MNT)  

# Summary of our Linear Regression Model
summary(linregmodel)


Call:
lm(formula = lifetime ~ . - broken, data = MNT)

Residuals:
    Min      1Q  Median      3Q     Max 
-59.388 -21.788   8.051  21.112  34.891 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)       59.3732039 10.3412622   5.741 1.25e-08 ***
pressureInd       -0.0031500  0.0416461  -0.076   0.9397    
moistureInd       -0.0173023  0.0830046  -0.208   0.8349    
temperatureInd    -0.0002769  0.0421330  -0.007   0.9948    
teamTeamB          1.5491323  1.9983947   0.775   0.4384    
teamTeamC         -3.4280411  2.0670679  -1.658   0.0976 .  
providerProvider2  0.8835691  2.2944030   0.385   0.7002    
providerProvider3 -9.4858216  2.3490911  -4.038 5.80e-05 ***
providerProvider4  1.8679357  2.3616268   0.791   0.4292    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 26.13 on 991 degrees of freedom
Multiple R-squared:  0.0339,	Adjusted R-squared:  0.0261 
F-statistic: 4.346 on 8 and 991 DF,  p-valu

It appears that the *providerProvider3* variable is statistically significant and has a negative effect on the lifetime of a piece. Here we cannot use a linear regression model because about 60% of the pieces haven't not broken yet. We don't know what the lifetime will be for pieces that haven't broken yet. All we know is that they are working until now.

### Survival Regression Model

As seen in the lecture, we need to create a survival regression model:

In [5]:
#install.packages("survival")

library(survival)

In [6]:
# choose the dependant variables to be used in the survival regression model with the Surv() function
dependantvars <- Surv(MNT$lifetime, MNT$broken) 

# Create the survival regression model
survreg <- survreg(dependantvars ~ pressureInd + moistureInd + temperatureInd + team + provider, dist="gaussian",data=MNT) 

summary(survreg) 


Call:
survreg(formula = dependantvars ~ pressureInd + moistureInd + 
    temperatureInd + team + provider, data = MNT, dist = "gaussian")
                      Value Std. Error       z      p
(Intercept)        8.04e+01   2.94e-01  273.57 <2e-16
pressureInd       -7.14e-04   1.22e-03   -0.59  0.557
moistureInd        6.01e-03   2.40e-03    2.51  0.012
temperatureInd    -1.04e-02   1.21e-03   -8.59 <2e-16
teamTeamB         -5.67e-02   5.88e-02   -0.96  0.335
teamTeamC         -6.22e+00   6.13e-02 -101.39 <2e-16
providerProvider2  1.25e+01   6.67e-02  187.46 <2e-16
providerProvider3 -1.44e+01   6.27e-02 -229.24 <2e-16
providerProvider4  7.92e+00   7.06e-02  112.23 <2e-16
Log(scale)        -7.43e-01   3.54e-02  -21.00 <2e-16

Scale= 0.476 

Gaussian distribution
Loglik(model)= -270.1   Loglik(intercept only)= -1557
	Chisq= 2573.75 on 8 degrees of freedom, p= 0 
Number of Newton-Raphson Iterations: 12 
n= 1000 


For example, moisture and temperature are statistically significant. Moisture has a positive effect on the lifetime, whereas the temperature has a negative effect.

Now let's make predictions on the estimated lifetime, using the survival regression model we just built. The predict function will output the median lifetime of each piece:

In [7]:
# Make predictions based on the model. We are setting the quantile to 50%. 
# Hence, we estimate the median lifetime as the expected moment of "death"
Ebreak <- predict(survreg, newdata=MNT, type="quantile", p=.5)

In [8]:
# Create a dataframe to store the ouput of Ebreak
Forecast <- data.frame(Ebreak)
head(Forecast)

,Ebreak
,<dbl>
1,87.82547
2,81.71139
3,79.58070
4,86.46185
5,79.74085
6,80.04827


Then we add the following columns to the predicted median expected lifetime dataframe:

* lifetime (= how a piece has been 'alive')
* broken (= is the piece broken or not?)
* remaining lifetime (=expected lifetime - lifetime to date)

In [9]:
# Add a column in the Forecast dataframe indicating the lifetime of the piece
Forecast$lifetime <- MNT$lifetime

# Add a column in the Forecast dataframe indicating whether or not the piece is broken
Forecast$broken <- MNT$broken

# Computed Expected Remaining Lifetime
Forecast$RemainingLT <- Forecast$Ebreak - MNT$lifetime     

head(Forecast)

,Ebreak,lifetime,broken,RemainingLT
,<dbl>,<int>,<int>,<dbl>
1,87.82547,56,0,31.8254667
2,81.71139,81,1,0.7113900
3,79.58070,60,0,19.5806969
4,86.46185,86,1,0.4618547
5,79.74085,34,0,45.7408455
6,80.04827,30,0,50.0482673


In order to take actions, we clean up the report by prioritizing (order by expected lifetime) and keeping only the non broken pieces:

In [10]:
# Orders the dataframe by Expected Remaining Lifetime
Forecast <- Forecast[order(Forecast$RemainingLT),]

# And keeps only those who are not broken yet
ActionsPriority <- Forecast[Forecast$broken==0,]

head(ActionsPriority)

,Ebreak,lifetime,broken,RemainingLT
,<dbl>,<int>,<int>,<dbl>
80,87.81934,88,0,-0.1806637
920,65.39920,65,0,0.3992047
138,81.62875,81,0,0.6287524
455,87.79644,87,0,0.7964376
917,85.91101,85,0,0.9110116
414,81.97309,81,0,0.9730926


### Questions

**1) If instead of considering the effects significant when the p-value is smaller than 0.05 (as during the videos), we set the threshold of significance to 0.01, which one of the following is correct?**

In [11]:
summary(survreg)


Call:
survreg(formula = dependantvars ~ pressureInd + moistureInd + 
    temperatureInd + team + provider, data = MNT, dist = "gaussian")
                      Value Std. Error       z      p
(Intercept)        8.04e+01   2.94e-01  273.57 <2e-16
pressureInd       -7.14e-04   1.22e-03   -0.59  0.557
moistureInd        6.01e-03   2.40e-03    2.51  0.012
temperatureInd    -1.04e-02   1.21e-03   -8.59 <2e-16
teamTeamB         -5.67e-02   5.88e-02   -0.96  0.335
teamTeamC         -6.22e+00   6.13e-02 -101.39 <2e-16
providerProvider2  1.25e+01   6.67e-02  187.46 <2e-16
providerProvider3 -1.44e+01   6.27e-02 -229.24 <2e-16
providerProvider4  7.92e+00   7.06e-02  112.23 <2e-16
Log(scale)        -7.43e-01   3.54e-02  -21.00 <2e-16

Scale= 0.476 

Gaussian distribution
Loglik(model)= -270.1   Loglik(intercept only)= -1557
	Chisq= 2573.75 on 8 degrees of freedom, p= 0 
Number of Newton-Raphson Iterations: 12 
n= 1000 


Pressure and Moisture are non-significant. Temperature is significant.

**2) If you use as explanatory variables only the Pressure, Moisture and Temperature indices (hence removing the teams and providers information), which one of the following is correct?**

In [12]:
survreg2 <- survreg(dependantvars ~ pressureInd + moistureInd + temperatureInd, dist="gaussian",data=MNT)

summary(survreg2) 


Call:
survreg(formula = dependantvars ~ pressureInd + moistureInd + 
    temperatureInd, data = MNT, dist = "gaussian")
                 Value Std. Error     z      p
(Intercept)    75.8525     5.9187 12.82 <2e-16
pressureInd     0.0228     0.0249  0.91   0.36
moistureInd     0.0394     0.0495  0.80   0.43
temperatureInd -0.0223     0.0251 -0.89   0.37
Log(scale)      2.3528     0.0340 69.19 <2e-16

Scale= 10.5 

Gaussian distribution
Loglik(model)= -1555.9   Loglik(intercept only)= -1557
	Chisq= 2.19 on 3 degrees of freedom, p= 0.53 
Number of Newton-Raphson Iterations: 6 
n= 1000 


For this model, Pressure, Moisture and Temperature indexes are greater than 0.05. Hence they are not statistically significant.

**3) What is the ID of the element that has the largest expected remaining lifetime?**

In [13]:
head( Forecast[order(-Forecast$RemainingLT),] )

,Ebreak,lifetime,broken,RemainingLT
,<dbl>,<int>,<int>,<dbl>
53,92.24826,1,0,91.24826
472,92.55563,2,0,90.55563
877,92.31880,2,0,90.31880
225,92.87914,3,0,89.87914
708,92.69553,4,0,88.69553
852,92.27872,5,0,87.27872


Item #53 has the largest expected remaining lifetime.